In [ ]:
# Load in Python Library
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sc
import xrft as xf
import matplotlib.colors as colors

In [ ]:
# Here are the parameters for the specific case this calculation is done for
theta = 1.8113*np.pi/180  # slope angle
N2 = 10**(-5)  # stratification
delta = 0.5  # wave magnitude
Vinf = 0.1  # interior velocity
f = 1e-4  # coriolis frequency
S2 = N2*np.tan(theta)**2/f**2  # squared Burger number
gamma = 0.5002284142579378  # stratification parameter
h = (f*Vinf)/(N2*gamma*np.tan(theta))  # boundary layer height
lmbd = (N2*gamma*np.tan(theta))/f  # geostrophic shear
fstar = np.cos(theta)*(f**2+N2*np.tan(theta)**2)**(0.5)  # modified inertial frequency
beta = np.cos(theta)*(1+S2)**(0.5)  # beta (portion of modified inertial frequency that modifies it)

In [ ]:
# Access the output file
file = "flow_fields_height_63.21485960107034_interior_velocity_0.1_visc_1.0e-5_Sinf_1.0000287024749517_gamma_0.5002284142579378_theta_1.8113_f_0.0001_N2_1.0e-5.nc"
ocean_data=xr.open_dataset("/glade/derecho/scratch/knudsenl/data/new_data/vinffixed/"+file)#.isel(time=slice(0,4000)) #i=103
seconds = ocean_data.time/pd.Timedelta("1s")
ocean_data = ocean_data.assign_coords(time = ('time',seconds.data))

In [ ]:
def z_change(t, z, x):  # advection by isopycnals
    z_osc = f*np.cos(theta)*delta*lmbd/(fstar**2)*(np.cos(fstar*t)-1)*(h-z)
    return z_osc

In [ ]:
# Caclulate the modified inertial period
inertial_period = ocean_data.time*fstar/(2*np.pi)
ocean_data_ip = ocean_data.assign_coords(intp = ('time',inertial_period.data))

In [ ]:
# Calculate the Quasi-Lagrangian v-velocity
ti = 200
tf = 800
zi = 40
xi = 512
v_lagrange = ocean_data.v.sel(xC=ocean_data.xC[xi]-z_change(ocean_data.time[ti:tf], 
                                                            ocean_data.zC[zi],ocean_data.xC[xi]), 
                              zC=ocean_data.zC[zi], time=ocean_data.time[ti:tf], method="nearest")
# Drops the dimensions so that time series can be plotted
v_lagrange = v_lagrange.drop_vars(['xC','yF','zC'])

In [ ]:
# Calculate the Quasi-Lagrangian v-velocity
ti = 200
tf = 800
zi = 40
xi = 256
u_lagrange = ocean_data.u.sel(xF=ocean_data.xF[xi]-z_change(ocean_data.time[ti:tf], 
                                                            ocean_data.zC[zi], ocean_data.xF[xi]), 
                              zC=ocean_data.zC[zi], time=ocean_data.time[ti:tf], method="nearest")
# Drops the dimensions so that time series can be plotted
u_lagrange = u_lagrange.drop_vars(['xF','yC','zC'])

In [ ]:
# Plot Quasi-Lagrangian u,v-velocities
plt.plot(u_lagrange.time*(fstar)/(2*np.pi),u_lagrange.values/u_lagrange.values[0],color = "r",label=f"$u/u_2$ at z={ocean_data_ip.zC[zi].values}")
plt.plot(v_lagrange.time*(fstar)/(2*np.pi),v_lagrange.values/u_lagrange.values[0],color = "k",alpha=0.8,label=f"$v/u_2$ at z={ocean_data_ip.zC[zi].values}")
plt.xlabel("Interial Period $(tf^*(2\pi)^{-1})$")
plt.yscale("symlog",linthresh=1)
plt.legend(loc="lower left")
# plt.savefig("S1_gamma_l_delta_0_5_quasi_lagrangian_time_series.pdf")
plt.show()

In [ ]:
# Plot background velocities
ti = 200
tf = 281
zi = 20
xi = 256
plt.plot(ocean_data.time[ti:tf]*(fstar)/(2*np.pi),ocean_data.ub.sel(xF=ocean_data.xC[xi],zC=ocean_data.zC[zi],time=ocean_data.time[ti:tf],method="nearest").values/ocean_data.vb.sel(xC=ocean_data.xC[xi],zC=ocean_data.zC[zi],time=ocean_data.time[ti],method="nearest").values,color = "r",label="$U/V_{10}$ at z="+str(ocean_data_ip.zC[zi].values))
plt.plot(ocean_data.time[ti:tf]*(fstar)/(2*np.pi),ocean_data.vb.sel(xC=ocean_data.xC[xi],zC=ocean_data.zC[zi],time=ocean_data.time[ti:tf],method="nearest").values/ocean_data.vb.sel(xC=ocean_data.xC[xi],zC=ocean_data.zC[zi],time=ocean_data.time[ti],method="nearest").values,color = "k",alpha=0.8,label="$V/V_{10}$ at z="+str(ocean_data_ip.zC[zi].values))
plt.xlim([10,14])
plt.xlabel("Interial Period $(tf^*(2\pi)^{-1})$")
# plt.yscale("symlog",linthresh=1)
plt.legend(loc="lower left",prop={'size':9})
# plt.gca().set_aspect(1.1)
# plt.figure(figsize=(12,8))
plt.text(9.4,1,"(c.)", ha="left",va="top")
plt.savefig("background_time_series_10_14.png")
# plt.xticks([5,7,9,11,13,15])
plt.show()

In [ ]:
# Plot the total power spectrum
transform_lagrange = xf.power_spectrum(u_lagrange,dim="time")
transform_lagrange.plot()
plt.xscale("symlog",linthresh=1e-10)

In [ ]:
# Plot power spectrum w/ quasi-lagrangian times series
fig, ax_dict = plt.subplot_mosaic([['(a.)','(b.)']],layout="constrained",figsize=(9,4))
ax = ax_dict['(b.)']
ax.plot(u_lagrange.time[:201]*(fstar)/(2*np.pi),u_lagrange.values[:201]/u_lagrange.values[0],color = "r",label="$u/u_{10}$ at z=16.02 m")
ax.plot(v_lagrange.time[:201]*(fstar)/(2*np.pi),v_lagrange.values[:201]/u_lagrange.values[0],color = "k",alpha=0.8,label="$v/u_{10}$ at z=16.02 m")
# plt.xlim([4.8,15.2])
ax.set_xlabel("Modified Inertial Period")
ax.set_xlim([10,20])
ax.set_yscale("symlog",linthresh=1)
ax.legend(loc="lower left")
ax.set_box_aspect(0.75)
ax.set_title("(b.)", fontfamily='serif', loc='left', fontsize='medium')
ax = ax_dict['(a.)']
ax.plot(transform_lagrange.freq_time[:]*2*np.pi,transform_lagrange[:],color="k")
ax.vlines(0.5*fstar,ymin=0,ymax=np.max(transform_lagrange)*1.05,color="b",alpha=0.5,linestyles="dotted",label="$f^*/2$")
ax.vlines(fstar,ymin=0,ymax=np.max(transform_lagrange)*1.05,color="k",alpha=0.5,linestyles="dotted",label="$f^*$")
ax.set_xlim([1e-05,transform_lagrange.freq_time[:].max()*2*np.pi])
ax.set_ylim([0,np.max(transform_lagrange)*1.05])
ax.set_box_aspect(0.9)
ax.set_title("(a.)", fontfamily='serif', loc='left', fontsize='medium')
ax.legend()
ax.set_xlabel("Frequency [s$^{-1}$]")
ax.set_xscale("log")
ax.set_ylabel("Power Spectrum Density [m$^3$ s$^{-2}$]")
# plt.savefig("S_1_gamma_l_delta_0_5_power_spectral_density_times_series.pdf")
plt.show()

In [ ]:
# Plot Power Spectrum Solely
plt.plot(transform_lagrange.freq_time[:]*2*np.pi,transform_lagrange[:],color="k")
plt.vlines(0.5*fstar,ymin=0,ymax=np.max(np.abs(transform_lagrange))*1.1,color="b",alpha=0.5,linestyles="dotted",label="$f^*/2$")
plt.vlines(fstar,ymin=0,ymax=np.max(np.abs(transform_lagrange))*1.1,color="k",alpha=0.5,linestyles="dotted",label="$f^*$")
plt.xlim([1.1218508e-05,0.001])
# plt.ylim([0.1,30])
plt.legend()
plt.xlabel("Frequency [s$^{-1}$]")
plt.xscale("log")
# plt.yscale("log")
plt.ylabel("Power Spectrum Density [m$^3$ s$^{-2}$]")
# plt.text(-0.0001,0.23,"(b.)", ha="left",va="top")
# plt.title("Quasi-Lagrangian Fourier Transform of u-velocity")
# plt.savefig("QL_fourier_transform.png")
plt.show()

In [ ]:
# None Quasi-Lagrangian velocities
ti = -200
tf = -1
plt.plot(ocean_data_ip.intp.isel(time=slice(ti, tf)), ocean_data_ip.u.isel(zC=64, yC=0, xF=128, time=slice(ti, tf))/ocean_data_ip.v.isel(zC=64, yF=0, xC=128, time=ti),color = "r",label=f"$u/v_5$ at z={ocean_data_ip.zC[64].values}")
plt.plot(ocean_data_ip.intp.isel(time=slice(ti, tf)), ocean_data_ip.v.isel(zC=64, yF=0, xC=128, time=slice(ti, tf))/ocean_data_ip.v.isel(zC=64, yF=0, xC=128, time=ti),color = "k",label=f"$v/v_5$ at z={ocean_data_ip.zC[64].values}")
# plt.xlim([4.8,15.2])
plt.xlabel("Interial Period $(tf^*(2\pi)^{-1})$")
plt.legend()
# plt.xticks([5,7,9,11,13,15])
plt.show()